In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
words = open("names.txt", "r").read().split()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars=sorted(list(set(''.join(words))))
stoi = {c:i+1 for i , c in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(stoi)
print(itos)



{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
block_size = 3 # context length: how many characters to take to predict the next character
X,Y=[],[]
for w in words[:5]:
    print(w)
    context=[0]*block_size # Creates an initial context of [0, 0, 0]. This serves as the starting context before actual characters from the word are added. Assuming 0 is a special padding token
    for ch in w + '.':# Iterate through each character in the word plus a stopping character ('.')
        ix =stoi[ch] # Convert character to its integer index using a predefined `stoi` dictionary
        X.append(context)
        Y.append(ix)
        print(X,Y)
        print(''.join(itos[i] for i in context),'---->',itos[ix])# Print the mapping from context to predicted character
        context=context[1:]+[ix] # Shift the context window by removing the first character and appending the new one
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
[[0, 0, 0]] [5]
... ----> e
[[0, 0, 0], [0, 0, 5]] [5, 13]
..e ----> m
[[0, 0, 0], [0, 0, 5], [0, 5, 13]] [5, 13, 13]
.em ----> m
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13]] [5, 13, 13, 1]
emm ----> a
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1]] [5, 13, 13, 1, 0]
mma ----> .
olivia
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0]] [5, 13, 13, 1, 0, 15]
... ----> o
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15]] [5, 13, 13, 1, 0, 15, 12]
..o ----> l
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12]] [5, 13, 13, 1, 0, 15, 12, 9]
.ol ----> i
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9]] [5, 13, 13, 1, 0, 15, 12, 9, 22]
oli ----> v
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9], [12, 9, 22]] [5, 13, 13, 1, 0, 15, 12, 9, 22, 9]
liv ----> i


In [6]:
X.shape,Y.shape


(torch.Size([32, 3]), torch.Size([32]))

In [7]:
X,Y

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1],
         [ 0,  0,  0],
         [ 0,  0, 15],
         [ 0, 15, 12],
         [15, 12,  9],
         [12,  9, 22],
         [ 9, 22,  9],
         [22,  9,  1],
         [ 0,  0,  0],
         [ 0,  0,  1],
         [ 0,  1, 22],
         [ 1, 22,  1],
         [ 0,  0,  0],
         [ 0,  0,  9],
         [ 0,  9, 19],
         [ 9, 19,  1],
         [19,  1,  2],
         [ 1,  2,  5],
         [ 2,  5, 12],
         [ 5, 12, 12],
         [12, 12,  1],
         [ 0,  0,  0],
         [ 0,  0, 19],
         [ 0, 19, 15],
         [19, 15, 16],
         [15, 16,  8],
         [16,  8,  9],
         [ 8,  9,  1]]),
 tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
          1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0]))

In [8]:
C= torch.randn((27,2))

In [ ]:
#F.one_hot(torch.tensor(5),num_classes=27).float() @ C

tensor([ 1.2623, -0.3006])